# Exploring Vector Database Operations in LangChain

## Install OpenAI, and LangChain dependencies

In [1]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 543.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 18.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.42
    Uninstalling langsmith-0.3.42:
      Successfully uninstalled langsmith-0.3.42
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy

## Install Chroma Vector DB and LangChain wrapper

In [2]:
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 6.9 MB/s eta 0:

## Setup Environment Variables

In [1]:
from google.colab import userdata
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [2]:
documents = ['Quantum mechanics describes the behavior of very small particles.',
 'Photosynthesis is the process by which green plants make food using sunlight.',
 "Shakespeare's plays are a testament to English literature.",
 'Artificial Intelligence aims to create machines that can think and learn.',
 'The pyramids of Egypt are historical monuments that have stood for thousands of years.']

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [3]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Vector Databases

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector database takes care of storing embedded data and performing vector search for you.

### Chroma Vector DB

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

In [4]:
# delete vector db if exists
!rm -rf ./chroma_db

### Create a Vector DB and persist on disk

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [5]:
from langchain_chroma import Chroma

# create empty vector DB
chroma_db = Chroma(collection_name='search_docs',
                   embedding_function=openai_embed_model,
                   persist_directory="./chroma_db")

We take some sample documents

In [6]:
documents

['Quantum mechanics describes the behavior of very small particles.',
 'Photosynthesis is the process by which green plants make food using sunlight.',
 "Shakespeare's plays are a testament to English literature.",
 'Artificial Intelligence aims to create machines that can think and learn.',
 'The pyramids of Egypt are historical monuments that have stood for thousands of years.']

We create document IDs to uniquely identify each document

In [7]:
ids = ['doc_'+str(i) for i in range(len(documents))]
ids

['doc_0', 'doc_1', 'doc_2', 'doc_3', 'doc_4']

Checking the Vector DB to see if its empty

In [8]:
chroma_db.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

### Adding documents to Vector DB

Here we take our texts, pass them through the Open AI embedder to get embeddings and add it to the Chroma Vector DB.

If you have documents in the LangChain `Document` format then you can use `add_documents` instead

In [9]:
chroma_db.add_texts(texts=documents, ids=ids)

['doc_0', 'doc_1', 'doc_2', 'doc_3', 'doc_4']

We check out Vector DB now to see these documents have been indexed successfully

In [10]:
chroma_db.get()

{'ids': ['doc_0', 'doc_1', 'doc_2', 'doc_3', 'doc_4'],
 'embeddings': None,
 'documents': ['Quantum mechanics describes the behavior of very small particles.',
  'Photosynthesis is the process by which green plants make food using sunlight.',
  "Shakespeare's plays are a testament to English literature.",
  'Artificial Intelligence aims to create machines that can think and learn.',
  'The pyramids of Egypt are historical monuments that have stood for thousands of years.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None, None, None, None, None]}

Run some search queries in our Vector DB

In [11]:
query = 'Tell me about AI'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_3', metadata={}, page_content='Artificial Intelligence aims to create machines that can think and learn.'),
  0.8804178833961487)]

In [12]:
query = 'Do you know about the pyramids?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_4', metadata={}, page_content='The pyramids of Egypt are historical monuments that have stood for thousands of years.'),
  0.8856476545333862)]

In [13]:
query = 'What is Biology?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_1', metadata={}, page_content='Photosynthesis is the process by which green plants make food using sunlight.'),
  1.509749174118042)]

### Adding more documents to our Vector DB

We can add new documents anytime to the vector DB as shown below

In [14]:
new_documents = [ 'Biology is the study of living organisms and their interactions with the environment.',
 'Music therapy can aid in the mental well-being of individuals.',
 'The Milky Way is just one of billions of galaxies in the universe.',
 'Economic theories help understand the distribution of resources in society.',
 'Yoga is an ancient practice that involves physical postures and meditation.']

In [15]:
new_ids = ['doc_'+str(i+len(ids)) for i in range(len(new_documents))]
new_ids

['doc_5', 'doc_6', 'doc_7', 'doc_8', 'doc_9']

In [16]:
chroma_db.add_texts(texts=new_documents, ids=new_ids)

['doc_5', 'doc_6', 'doc_7', 'doc_8', 'doc_9']

In [17]:
chroma_db.get()

{'ids': ['doc_0',
  'doc_1',
  'doc_2',
  'doc_3',
  'doc_4',
  'doc_5',
  'doc_6',
  'doc_7',
  'doc_8',
  'doc_9'],
 'embeddings': None,
 'documents': ['Quantum mechanics describes the behavior of very small particles.',
  'Photosynthesis is the process by which green plants make food using sunlight.',
  "Shakespeare's plays are a testament to English literature.",
  'Artificial Intelligence aims to create machines that can think and learn.',
  'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
  'Biology is the study of living organisms and their interactions with the environment.',
  'Music therapy can aid in the mental well-being of individuals.',
  'The Milky Way is just one of billions of galaxies in the universe.',
  'Economic theories help understand the distribution of resources in society.',
  'Yoga is an ancient practice that involves physical postures and meditation.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': 

In [18]:
query = 'What is Biology?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_5', metadata={}, page_content='Biology is the study of living organisms and their interactions with the environment.'),
  0.6787382960319519)]

### Updating documents in the Vector DB

While building toward a real application, you want to go beyond adding data, and also update and delete data.

Chroma has users provide ids to simplify the bookkeeping here and update documents as shown below using the `update_documents`function

In [19]:
chroma_db.get(['doc_3'])

{'ids': ['doc_3'],
 'embeddings': None,
 'documents': ['Artificial Intelligence aims to create machines that can think and learn.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None]}

In [20]:
from langchain_core.documents import Document

ids = ['doc_3']
texts = ['AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.']
documents = [Document(page_content=text, metadata={'doc': id})
                for id, text in zip(ids,texts)]
documents

[Document(metadata={'doc': 'doc_3'}, page_content='AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.')]

In [21]:
chroma_db.update_documents(ids=ids,documents=documents)

In [22]:
chroma_db.get(['doc_3'])

{'ids': ['doc_3'],
 'embeddings': None,
 'documents': ['AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'doc': 'doc_3'}]}

In [23]:
query = 'What is AI?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_3', metadata={'doc': 'doc_3'}, page_content='AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.'),
  0.563330352306366)]

### Deleting documents in the Vector DB

Chroma has users provide ids to simplify the bookkeeping here and delete documents as shown below using the `delete`function

In [24]:
chroma_db.delete(['doc_9'])

In [25]:
chroma_db.get()

{'ids': ['doc_0',
  'doc_1',
  'doc_2',
  'doc_3',
  'doc_4',
  'doc_5',
  'doc_6',
  'doc_7',
  'doc_8'],
 'embeddings': None,
 'documents': ['Quantum mechanics describes the behavior of very small particles.',
  'Photosynthesis is the process by which green plants make food using sunlight.',
  "Shakespeare's plays are a testament to English literature.",
  'AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.',
  'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
  'Biology is the study of living organisms and their interactions with the environment.',
  'Music therapy can aid in the mental well-being of individuals.',
  'The Milky Way is just one of billions of galaxies in the universe.',
  'Economic theories help understand the distribution of resources in society.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None,
  None,
  None,
  {'doc':

### Load Vector DB from disk

Once we have saved DB to disk, we can load it up anytime and connect to it and run queries as shown below

In [26]:
# load from disk
db = Chroma(persist_directory="./chroma_db",
            embedding_function=openai_embed_model,
            collection_name='search_docs')

query = 'What is AI?'
docs = db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_3', metadata={'doc': 'doc_3'}, page_content='AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.'),
  0.5632575154304504)]